<a href="https://colab.research.google.com/github/Himanshid914/SmartRead/blob/main/SmartRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================ Install Required Packages ============================
!pip install -q streamlit pyngrok scikit-learn pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:

# ============================ Upload the Dataset ============================
from google.colab import files
uploaded = files.upload()  # Upload 'goodreads_data.csv'



Saving goodreads_data.csv to goodreads_data.csv


In [3]:
# ============================ Save Streamlit App Code ============================
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
books = pd.read_csv("goodreads_data.csv")
books['Description'] = books['Description'].fillna('')

# TF-IDF + Cosine Similarity
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['Description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Recommendation logic
def get_recommendations(book_title, cosine_sim=cosine_sim):
    if book_title not in books['Book'].values:
        return ["Book title not found in the dataset"]
    idx = books[books['Book'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    book_indices = [i[0] for i in sim_scores]
    return books['Book'].iloc[book_indices]

# UI
st.title("📚 Book Recommender")
book_input = st.sidebar.text_input("Enter a Book Title")
if st.sidebar.button("Recommend"):
    if book_input:
        results = get_recommendations(book_input)
        st.subheader("Recommended Books:")
        for i, book in enumerate(results):
            st.write(f"{i+1}. {book}")
    else:
        st.warning("Please enter a valid book title.")


Writing app.py


In [6]:
import os
import time
import threading
from pyngrok import ngrok, conf

# Kill any previous processes
!fuser -k 8501/tcp || true

# Set your ngrok auth token here
conf.get_default().auth_token = "2vtlj5b0zBzKDzePFETEAP8aqds_7xCshPKQZzmujKxYZ4NTX"

# Start Streamlit app in a thread
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

# Wait a few seconds for Streamlit to boot
time.sleep(10)

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🚀 Your app is live at: {public_url}")


8501/tcp:             2980


🚀 Your app is live at: NgrokTunnel: "https://635f-34-82-195-131.ngrok-free.app" -> "http://localhost:8501"
